<a href="https://colab.research.google.com/github/VanessaIri/Proyecto-1-ASB/blob/main/Proyecto_1_ASB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importar las librerías correspondientes
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from pandas import read_csv

In [ ]:
#código para leer archivo csv
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#obtener el archivo
downloaded=drive.CreateFile({'id':'1TIWdv3yskJWkLj4-1Z_Y8EDC4tyjItQh'})
downloaded.GetContentFile('Datosp1.csv')

In [ ]:
#leer el archivo en csv indicando que no hay encabezados
data = read_csv('Datosp1.csv',header=None, delimiter=',')

In [ ]:
data.shape #corroborar tamaño de matriz de datos

In [ ]:
#Por gusto personal se transpone la matriz de datos
datos=data.to_numpy()
Datos=np.transpose(datos)
Datos.shape[1]

In [ ]:
#Por conveniencia de cálculos se realiza una matriz de 0s con dos columnas más grande que los datos
Data=np.zeros((48,370))
Data[: , 1:369]=Datos #se sustituyen los 0s de la columna 1 a la 368 por los datos dados, columna 0 y 369 se quedan en 0s 
Data

In [ ]:
Data.shape[1]

In [ ]:
#Autocorrelación de la matriz de mediciones
n = 368
R = np.zeros((48,2*n-1)) #se inicializa una matriz de 0s para guardar las autocorrelaciones
for k in range(-n+1, n): #cálculo de autocorrelaciones para toda la matriz de datos, 
  L1=Data[: , 1:-1-abs(k)]    #basado en el estimador de Childers se obtiene cada una de las 48 autocorrelaciones
  L2=Data[: , 1+abs(k):-1]
  R[: , 367+k] = R[: , 367+k] + np.multiply(np.sum(np.multiply(L1,L2), axis=1), 1/n)  #matriz de autocorrelaciones
 

In [ ]:
R[0] #ejemplo de primer autocorrelación

In [ ]:
#Graficas de (Autocorelacion), obtiene las 48 autocorrelaciones en subfiguras
fig= plt.figure(figsize=(40, 70))
fig.tight_layout()

for i in range(1,49):
  x=np.arange(-367,368)            #Lags
  y=R[i-1]                         #Valores de autocorrelacion
  ax= plt.subplot(12,4,i)
  ax.plot(x,y, color='magenta')
  ax.set_xlabel('Lag', fontsize=10)
  ax.set_ylabel('Valor', fontsize=10)
  ax.set_title('Autocorrelación de Medición '+str(i), fontsize=14)


In [ ]:
# Cálculo de la Autocorrelación Promedio 
r=np.zeros((735)) 
for i in range(0,48):
  r=r+R[i]   #suma los 48 vectores de autocorrelacion con sus 735 datos correspondientes

rr=r/48  #autocorrelacion promedio 


In [ ]:
#Grafica de autocorrelacion promedio
plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(list(range(-367,368)),rr)
plt.ylabel('Magnitud')
plt.xlabel('Lag')
plt.title('Autocorrelación Promedio')
plt.show()

In [ ]:
#Espectro de la autocorrelación promedio
Srr= abs(np.fft.fftshift(np.fft.fft(rr)))  #transformada de fourier para calcular espectro

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(np.multiply((160/735),np.array(range(-367,368))),Srr) #grafica para el rango de frecuencias correspondiente -80 a 80
plt.ylabel('Magnitud')
plt.xlabel('Frecuencia (Hz)')
plt.title('Espectro de autocorrelación promedio')
#plt.xlim(-12,12)
plt.show()

In [ ]:
# medición promedio
x=np.zeros((370))
for i in range(0,48):
  x=x+Data[i]

xProm=x/48


In [ ]:
#autocorrelación de medición promedio
n = 368
RR = np.zeros((2*n-1))
for k in range(-n+1, n):    #se emplea el estimador de childers
  l1=xProm[1:-1-abs(k)]
  l2=xProm[1+abs(k):-1]
  RR[367+k] = RR[367+k] + np.multiply(np.sum(np.multiply(l1,l2)), 1/n)

In [ ]:
#Gráfica de autocorrelación de medición promedio
plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(list(range(-367,368)),RR)
plt.ylabel('Magnitud')
plt.xlabel('Lag')
plt.title('Autocorrelación de medición promedio')
plt.show()

In [ ]:
#Espectro de la autocorrelación con medición promedio
SxProm= abs(np.fft.fftshift(np.fft.fft(RR)))   #Transformada de Fourier

plt.figure(figsize=(12, 12))
plt.subplot(2, 1, 1)
plt.plot(np.multiply((160/735),np.array(range(-367,368))),SxProm) #grafica para el rango de frecuencias correspondiente -80 a 80
plt.ylabel('Magnitud')
plt.xlabel('Frecuencia (Hz)')
plt.title('Espectro de autocorrelación con medición promedio')
#plt.xticks(ticks=list(range(-80,80)))
#plt.xlim(-20,20)
plt.show()


In [ ]:
#Comparación de los espectros obtenidos
plt.figure(figsize=(15, 15))
plt.subplot(2, 1, 1)
plt.plot(np.multiply((160/735),np.array(range(-367,368))),SxProm,'c') #espectro de la autocorrelación del promedio de los datos
plt.plot(np.multiply((160/735),np.array(range(-367,368))),Srr,'m') #espectro de la autocorrelación promedio
plt.ylabel('Magnitud')
plt.xlabel('Frecuencia (Hz)')
plt.title('Espectros de autocorrelación')
#plt.xticks(ticks=list(range(-80,80)))
#plt.xlim(-15,15)  #Acercamiento a los datos
plt.show()


https://fisiologia.facmed.unam.mx/index.php/fisiologia-de-la-actividad-electrica-del-cerebro/

http://neurofisiologiagranada.com/eeg/eeg-comoselee.htm 

https://www.acnweb.org/guia/g7cap17.pdf

Elegiria la 2 ya que parece ser que se cancela tipo ruido y queda una senal maslimpia mas facil de observar los picos altos de potencia
El rango va de -13 a 13 aproximadamente lo que indica que la frecuencia es entonces de aprox 26 que son ondas beta, se debe de encontrar en vigilia con
actividad mental y preferentemente se observa en regiones anteriores (frontales). Su distribución es fronto-central y puede haber reactividad ante estímulos táctiles y actividad motora de las extremidades contralaterales. 